In [1]:
import csv
from graphviz import Digraph

In [2]:
class Vertex:
    def __init__(self, num, attributes):
        self.num = num
        self.attributes = attributes
        self.toVertices = []
        self.fromVertices = []
        self.group = None
        self.groupnext = None
        self.groupprev = None
        self.lastdividenum = -1
class Group:
    def __init__(self, num):
        self.num = num
        self.vertexList = None               
        self.isinlist = False
        self.divideGroup = None
        self.divideNum = None


In [3]:
def getInputType1(vfile,efile,id1): #Eg: files in studentTestData
    
    #Vertices
    with open(vfile, 'r') as f:
        reader = csv.reader(f)
        vlist = list(reader)
    print(vlist)
    allAttr = {}
    allIds = {}
    for i in range(len(vlist[0])):
        allAttr[vlist[0][i]]=i
    if id1 not in allAttr:
        print(id1, 'not found')
        return
    else:
        idcol = allAttr[id1]
    V = []
    for i in range(1,len(vlist)):
        if len(vlist[i])==len(vlist[0]):
            allIds[vlist[i][idcol]]=i-1
            V.append(Vertex(i-1,vlist[i]))
    
    
    #Edges
    with open(efile, 'r') as f:
        reader = csv.reader(f)
        elist = list(reader)
    print(elist)
    #for i in range(len(elist[0])):
    col = [allIds[v] if v in allIds else None for v in elist[0]]
    for row in elist[1:]:
        if len(row)>0:
            v = allIds[row[0]]
            for i in range(1,len(row)):
                if row[i]=='1':
                    V[v].toVertices.append(col[i])
                    V[col[i]].fromVertices.append(v)
    '''
    for v in V:
        print(v.num)
        for v2 in v.fromVertices:
            print('\t',v2)
        for v2 in v.toVertices:
            print('\t\t',v2)
    '''
    return V,allAttr
            

In [4]:
V,allAttr = getInputType1('studentTestData/graph.csv','studentTestData/Relation.csv','Vertex')
attr =['Department']#,'Gender']


[['Vertex', 'Type', 'ID', 'Department', 'Gender'], ['v1', 'Student', '1', 'CS', 'Male'], ['v2', 'Student', '2', 'CS', 'Male'], ['v3', 'Student', '3', 'CS', 'Female'], ['v4', 'Student', '4', 'Booth', 'Male'], ['v5', 'Student', '5', 'Booth', 'Male'], ['v6', 'Student', '6', 'Booth', 'Female'], ['v7', 'Student', '7', 'Booth', 'Male'], ['v8', 'Student', '8', 'Booth', 'Male']]
[['', 'v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8'], ['v1', '', '', '', '', '', '1', '', ''], ['v2', '', '', '', '', '', '', '', ''], ['v3', '', '', '', '1', '1', '', '', ''], ['v4', '', '', '1', '', '1', '', '', ''], ['v5', '', '', '1', '1', '', '1', '', ''], ['v6', '1', '', '', '', '1', '', '1', '1'], ['v7', '', '', '', '', '', '1', '', ''], ['v8', '', '', '', '', '', '1', '', '']]


In [5]:
def getAComp(V,allAttr,attr):
    attr1 = [allAttr[a] for a in attr]
    Vs = sorted(V, key= lambda v : [v.attributes[i] for i in attr1])
    attrVal = None
    G = []
    for j in range(len(Vs)):
        if [Vs[j].attributes[i] for i in attr1]==attrVal:
            gnum = len(G)-1
            Vs[j].group = G[gnum]
            Vs[j].groupnext = G[gnum].vertexList
            if G[gnum].vertexList!=None:
                G[gnum].vertexList.groupprev = Vs[j]
            G[gnum].vertexList = Vs[j]
        else:
            gnum = len(G)
            G.append(Group(gnum))
            Vs[j].group = G[gnum]
            G[gnum].vertexList = Vs[j]
            attrVal = [Vs[j].attributes[i] for i in attr1]
    return(G)

In [6]:
def SNAP(V,allAttr,attr):
    G = getAComp(V,allAttr,attr)
    glist = None
    for g in G:
        glist = (g,glist)
        g.isinlist = True
    
    divideNum = 0
    gnum = len(G)
    while glist!=None:
        g = glist[0]
        glist = glist[1]
        g.isinlist = False
        
        gvlist = None
        v = g.vertexList
        while v!=None:
            gvlist = (v,gvlist)
            v = v.groupnext
        
        
        #divide using 'fromVertices' of g
        
        divideNum+=1
        dividelist=None
        
        gv = gvlist
        while gv!=None:
            v = gv[0]
            gv = gv[1]
            for v2num in v.fromVertices:
                v2 = V[v2num]
                if v2.lastdividenum!=divideNum:
                    g2 = v2.group
                    #Make new group if not exists
                    if g2.divideNum!=divideNum:
                        dividelist = (g2,dividelist)
                        G.append(Group(gnum))
                        g2.divideGroup = G[gnum]
                        g2.divideNum = divideNum
                        gnum+=1
                    g3 = g2.divideGroup
                    #Remove from Previous Group
                    if v2.groupprev==None:
                        g2.vertexList = v2.groupnext
                    else:
                        v2.groupprev.groupnext = v2.groupnext
                    if v2.groupnext!=None:
                        v2.groupnext.groupprev = v2.groupprev
                    #Add to new group
                    v2.group = g3
                    if g3.vertexList!=None:
                        g3.vertexList.groupprev = v2
                    v2.groupnext = g3.vertexList
                    v2.groupprev = None
                    g3.vertexList = v2
                    v2.lastdividenum = divideNum
        #Add groups to glist
        while dividelist!=None:
            g2 = dividelist[0]
            dividelist=dividelist[1]
            if g2.vertexList!=None:
                if not g2.isinlist:
                    g2.isinlist=True
                    glist = (g2,glist)
                g3 = g2.divideGroup
                g3.isinlist = True
                glist = (g3,glist)
            elif g2.isinlist:
                glist = (g2.divideGroup,glist)
                g2.divideGroup.isinlist=True
        
        #Check if g got divided
        if g.divideNum == divideNum:
            if g.vertexList==None:
                g = g.divideGroup
            else:
                continue
        
        #divide using 'toVertices' of g
        
        divideNum+=1
        dividelist=None
        
        gv = gvlist
        while gv!=None:
            v = gv[0]
            gv = gv[1]
            for v2num in v.toVertices:
                v2 = V[v2num]
                if v2.lastdividenum!=divideNum:
                    g2 = v2.group
                    #Make new group if not exists
                    if g2.divideNum!=divideNum:
                        dividelist = (g2,dividelist)
                        G.append(Group(gnum))
                        g2.divideGroup = G[gnum]
                        g2.divideNum = divideNum
                        gnum+=1
                    g3 = g2.divideGroup
                    #Remove from Previous Group
                    if v2.groupprev==None:
                        g2.vertexList = v2.groupnext
                    else:
                        v2.groupprev.groupnext = v2.groupnext
                    if v2.groupnext!=None:
                        v2.groupnext.groupprev = v2.groupprev
                    #Add to new group
                    v2.group = g3
                    if g3.vertexList!=None:
                        g3.vertexList.groupprev = v2
                    v2.groupnext = g3.vertexList
                    v2.groupprev = None
                    g3.vertexList = v2
                    v2.lastdividenum = divideNum
        #Add groups to glist
        while dividelist!=None:
            g2 = dividelist[0]
            dividelist=dividelist[1]
            if g2.vertexList!=None:
                if not g2.isinlist:
                    g2.isinlist=True
                    glist = (g2,glist)
                g3 = g2.divideGroup
                g3.isinlist = True
                glist = (g3,glist)
            elif g2.isinlist:
                glist = (g2.divideGroup,glist)
                g2.divideGroup.isinlist=True
        

In [7]:
SNAP(V,allAttr,attr)
print(sorted([(v.num,v.group.num) for v in V],key=lambda v: v[1]))

[(1, 1), (0, 17), (6, 19), (7, 19), (5, 27), (3, 30), (2, 34), (4, 35)]


In [8]:
dot1 = Digraph(comment='Graph1')

glist = {}
colors = ['red','blue','green','yellow','cyan','magenta','Purple','grey','tan','SeaGreen',
          'chocolate','salmon','YellowGreen','Lavender','Olive','Orange','Violet','PeachPuff']
for k in range(len(V)):
    if V[k].group.num not in glist:
        glist[V[k].group.num]=len(glist)
    i = glist[V[k].group.num]
    color = colors[i if i<len(colors) else len(colors)-1]
    
    dot1.node(str(V[k].num),str(V[k].attributes),color = color,style='filled')
    for j in V[k].toVertices:
        dot1.edge(str(V[k].num),str(j))

dot1.render('testOutput/Graph1')

'testOutput/Graph1.pdf'